### Import the Python packages

In [128]:
import rdflib
import os, sys, json, re
import pandas as pd
import networkx as nx
from copy import deepcopy

### Read the RXNORM and ATC terminologies in Turtle format (downloaded from BioPortal website)

In [2]:
rxnorm = rdflib.Graph()
rxnorm.parse("RXNORM.ttl", format="turtle") #http://bioportal.bioontology.org/ontologies/RXNORM
atc = rdflib.Graph()
atc.parse("ATC.ttl", format="turtle") #http://bioportal.bioontology.org/ontologies/ATC

<Graph identifier=Nb17982addba24159b35cfb9aa670fd0e (<class 'rdflib.graph.Graph'>)>

In [62]:
# This is not required for this problem, since we start by selecting a set of ATC drug classes and navigate the graph, 
# but it is good to show RDF Lib allows this ...
comb = rxnorm + atc

### Exploring the RXNORM and ATC terminologies -- size, and predicates used

In [68]:
print len(rxnorm)
print len(atc)
print len(comb)

2231734
62567
2293792


In [49]:
for m in atc:
    print m
    break

(rdflib.term.URIRef(u'http://purl.bioontology.org/ontology/UATC/H01AC04'), rdflib.term.URIRef(u'http://bioportal.bioontology.org/ontologies/umls/tui'), rdflib.term.Literal(u'T116', datatype=rdflib.term.URIRef(u'http://www.w3.org/2001/XMLSchema#string')))


In [4]:
atcpreds = set([m for m in atc.predicates()])
rxnormpreds = set([m for m in rxnorm.predicates()])

In [6]:
for m in atcpreds: print m

http://purl.bioontology.org/ontology/UATC/IS_DRUG_CLASS
http://bioportal.bioontology.org/ontologies/umls/cui
http://www.w3.org/2002/07/owl#imports
http://www.w3.org/1999/02/22-rdf-syntax-ns#type
http://www.w3.org/2004/02/skos/core#prefLabel
http://www.w3.org/2004/02/skos/core#notation
http://www.w3.org/2004/02/skos/core#altLabel
http://bioportal.bioontology.org/ontologies/umls/hasSTY
http://www.w3.org/2000/01/rdf-schema#comment
http://www.w3.org/2002/07/owl#versionInfo
http://www.w3.org/2000/01/rdf-schema#subClassOf
http://purl.bioontology.org/ontology/UATC/ATC_LEVEL
http://www.w3.org/2000/01/rdf-schema#label
http://bioportal.bioontology.org/ontologies/umls/tui


In [48]:
for m in rxnormpreds: print m

http://purl.bioontology.org/ontology/RXNORM/AMBIGUITY_FLAG
http://purl.bioontology.org/ontology/RXNORM/RXN_IN_EXPRESSED_FLAG
http://purl.bioontology.org/ontology/RXNORM/RXN_QUANTITY
http://www.w3.org/1999/02/22-rdf-syntax-ns#type
http://purl.bioontology.org/ontology/RXNORM/has_doseformgroup
http://purl.bioontology.org/ontology/RXNORM/has_ingredient
http://purl.bioontology.org/ontology/RXNORM/contained_in
http://www.w3.org/2004/02/skos/core#altLabel
http://bioportal.bioontology.org/ontologies/umls/hasSTY
http://purl.bioontology.org/ontology/RXNORM/RXN_STRENGTH
http://www.w3.org/2000/01/rdf-schema#subClassOf
http://www.w3.org/2004/02/skos/core#notation
http://www.w3.org/2002/07/owl#versionInfo
http://bioportal.bioontology.org/ontologies/umls/cui
http://purl.bioontology.org/ontology/RXNORM/has_quantified_form
http://purl.bioontology.org/ontology/RXNORM/part_of
http://purl.bioontology.org/ontology/RXNORM/RXN_AVAILABLE_STRENGTH
http://www.w3.org/2004/02/skos/core#prefLabel
http://www.w3.org

### Based on the predicates -- Helper functions to extract content (drugs, CUIs, dose forms, etc.) from the RXNORM and ATC terminologies, related to Opioid-Drugs

In [131]:
UMLS_CUI_PRED = rdflib.term.URIRef("http://bioportal.bioontology.org/ontologies/umls/cui")
RXNORM_CUI_PRED = rdflib.term.URIRef("http://purl.bioontology.org/ontology/RXNORM/RXCUI")

def get_sample_subjects(pred, graph, size=10):
    ccount = 0
    subjects = []
    for s in graph.subjects(predicate=pred):
        ccount += 1
        subjects.append(s)
        if ccount > size: break
    return subjects

def get_sample_objects(pred, graph, size=10):
    ccount = 0
    objects = []
    for o in graph.objects(predicate=pred):
        ccount += 1
        objects.append(o)
        if ccount > size: break
    return objects

def get_children(uri, graph):
    children = [m for m in graph.subjects(predicate=rdflib.RDFS.subClassOf, object=uri)]
    return children

def get_descendants(uri, graph, level=None):
    # This retrieves all descendants as a list
    descendants = set([])
    parents = [uri]
    lccount = 0
    level = 10 if not level else level
    while True:
        lccount += 1
        if lccount > level: break
        children = set([])
        for k in parents: 
            children = children.union(set(get_children(k, graph)))
        parents = children
        descendants = descendants.union(children)
        if len(parents) == 0: break
    return descendants

def get_descendants_hier(uri, graph, level=None):
    # This retrieves all descendants as a directed tree/network
    descendants = nx.DiGraph()
    parents = [uri]
    lccount = 0
    level = 10 if not level else level
    descendants.add_node(uri)
    while True:
        lccount += 1
        if lccount > level: break
        children = set([])
        for k in parents: 
            _children = set(get_children(k, graph))
            for m in _children: descendants.add_edge(k, m, etype="subclassOf")
            children = children.union(_children)
        parents = children
        if len(parents) == 0: break
    return descendants

def get_preferred_label(uri, graph):
    labels = [k[1].n3() for k in graph.preferredLabel(uri)]
    return labels

def get_objects(uri, pred, graph):
    objects = []
    for o in graph.objects(subject=uri, predicate=pred): objects.append(o)
    return objects

def get_subjects(uri, pred, graph):
    subjects = []
    for s in graph.subjects(object=uri, predicate=pred): subjects.append(s)
    return subjects

def get_cui_codes(uri, graph):
    objects = get_objects(uri, UMLS_CUI_PRED, graph)
    return objects

def get_rxcui_codes(uri, graph):
    objects = get_objects(uri, RXNORM_CUI_PRED, graph)
    return objects

def get_rxnorm_ids(uri, graph):
    #get rxnorm ids likned to a cui
    subjects = get_subjects(uri, UMLS_CUI_PRED, graph)
    return subjects

first_cap_re = re.compile('(.)([A-Z][a-z]+)')
all_cap_re = re.compile('([a-z0-9])([A-Z])')

def parse_camel_case(name):
    s1 = first_cap_re.sub(r'\1_\2', name)
    return all_cap_re.sub(r'\1_\2', s1).lower()

def parse_uri_token(token):
    prop = parse_camel_case(token)
    pparts = re.split("[-_]", prop)
    name = " ".join([x.title() for x in pparts])
    return name

def parseURI(uri):
    nparts = re.split("[#:/]", uri)
    name = parse_uri_token(nparts[len(nparts)-1])
    return name

### Testing these helper functions 

In [52]:
list(get_descendants(rdflib.term.URIRef('http://purl.bioontology.org/ontology/UATC/N02A'), atc))[0:10]

[rdflib.term.URIRef(u'http://purl.bioontology.org/ontology/UATC/N02AC'),
 rdflib.term.URIRef(u'http://purl.bioontology.org/ontology/UATC/N02AB'),
 rdflib.term.URIRef(u'http://purl.bioontology.org/ontology/UATC/N02AA'),
 rdflib.term.URIRef(u'http://purl.bioontology.org/ontology/UATC/N02AG'),
 rdflib.term.URIRef(u'http://purl.bioontology.org/ontology/UATC/N02AF'),
 rdflib.term.URIRef(u'http://purl.bioontology.org/ontology/UATC/N02AE'),
 rdflib.term.URIRef(u'http://purl.bioontology.org/ontology/UATC/N02AD'),
 rdflib.term.URIRef(u'http://purl.bioontology.org/ontology/UATC/N02AJ'),
 rdflib.term.URIRef(u'http://purl.bioontology.org/ontology/UATC/N02AJ09'),
 rdflib.term.URIRef(u'http://purl.bioontology.org/ontology/UATC/N02AJ08')]

In [10]:
get_preferred_label(rdflib.term.URIRef('http://purl.bioontology.org/ontology/UATC/N02A'), atc)

[u'"OPIOID ANALGESICS"@en']

In [11]:
get_sample_objects(rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#label'), atc)

[rdflib.term.Literal(u'ATC LEVEL'),
 rdflib.term.Literal(u'ATC'),
 rdflib.term.Literal(u'Is Drug Class'),
 rdflib.term.Literal(u'Semantic type UMLS property')]

In [39]:
str(get_cui_codes(rdflib.term.URIRef(u'http://purl.bioontology.org/ontology/UATC/N02AA'), atc)[0])

'C3653996'

In [43]:
# weirdly you need to keep the datatype in here, otherwise the query does not work ... (datatype info is retrieved from atc graph)
get_rxnorm_ids(rdflib.term.Literal(u'C3668758', datatype=rdflib.term.URIRef(u'http://www.w3.org/2001/XMLSchema#string')), rxnorm)

[rdflib.term.URIRef(u'http://purl.bioontology.org/ontology/RXNORM/1441383')]

In [53]:
list(get_subjects(rdflib.term.URIRef(u'http://purl.bioontology.org/ontology/RXNORM/316965'), 
             rdflib.term.URIRef(u'http://purl.bioontology.org/ontology/RXNORM/has_dose_form'), rxnorm))[0:10]

[rdflib.term.URIRef(u'http://purl.bioontology.org/ontology/RXNORM/375556'),
 rdflib.term.URIRef(u'http://purl.bioontology.org/ontology/RXNORM/313363'),
 rdflib.term.URIRef(u'http://purl.bioontology.org/ontology/RXNORM/245925'),
 rdflib.term.URIRef(u'http://purl.bioontology.org/ontology/RXNORM/854846'),
 rdflib.term.URIRef(u'http://purl.bioontology.org/ontology/RXNORM/1722357'),
 rdflib.term.URIRef(u'http://purl.bioontology.org/ontology/RXNORM/858264'),
 rdflib.term.URIRef(u'http://purl.bioontology.org/ontology/RXNORM/1918227'),
 rdflib.term.URIRef(u'http://purl.bioontology.org/ontology/RXNORM/1242231'),
 rdflib.term.URIRef(u'http://purl.bioontology.org/ontology/RXNORM/968643'),
 rdflib.term.URIRef(u'http://purl.bioontology.org/ontology/RXNORM/371986')]

In [40]:
get_cui_codes(rdflib.term.URIRef(u'http://purl.bioontology.org/ontology/RXNORM/1441383'), rxnorm)

[rdflib.term.Literal(u'C3668758', datatype=rdflib.term.URIRef(u'http://www.w3.org/2001/XMLSchema#string'))]

In [67]:
# weirdly you need to keep the datatype in here, otherwise the query does not work ... (datatype info is retrieved from atc graph)
get_rxnorm_ids(rdflib.term.Literal(u'C4282509', datatype=rdflib.term.URIRef(u'http://www.w3.org/2001/XMLSchema#string')), comb)

[rdflib.term.URIRef(u'http://purl.bioontology.org/ontology/RXNORM/1806700'),
 rdflib.term.URIRef(u'http://purl.bioontology.org/ontology/UATC/N02AA56')]

### This is the main code to generate an OPIOID related Drug Network with CUIs, RxNorm Ids, ATC ids, dose etc.

In [248]:
def get_opioid_drugs_atc(atc_hiercs):
    # I am generating a networkx DiGraph data structure so as to add other type of URIs (RxNorm IDs etc.)
    #return Cui, label, iris
    opioid_drugs = nx.DiGraph()
    for a in atc_hiercs:
        _drugs = get_descendants_hier(rdflib.term.URIRef(a), atc)
        for k in _drugs.nodes(): 
            opioid_drugs.add_node(k.n3(), rdfid=k, label=get_preferred_label(k, atc)[0], ntype="atc_id")
            cui_codes = get_cui_codes(k, atc)
            for m in cui_codes:
                opioid_drugs.add_node(str(m), rdfid=m, label=str(m), ntype="cui_id")
                opioid_drugs.add_edge(k.n3(), str(m), etype="has_cui")
        for k in _drugs.edges():
            opioid_drugs.add_edge(k[0].n3(), k[1].n3(), etype="subclassOf")
    return opioid_drugs

def extend_rxnorm_network(opioid_drugs, ntype="rxnorm_id", extype="rxnorm_id_ext"):
    ### Extend the Opioid Drug Network with generic RXNorm IDs to include other drug formulations 
    ### (E.g. morphine to morphine sulphate 40mg to morphine sulphate 40mg oral tablet 
    ### (basically a 2-degree hop on the Rxnorm graph)
    opioid_drugs_ext = deepcopy(opioid_drugs)
    for m in opioid_drugs.nodes():
        if opioid_drugs.node[m]["ntype"] == ntype: 
            #print opioid_drugs.node[m]
            rxcuis = []
            rxauis = []
            for k in rxnorm.predicate_objects(subject=opioid_drugs.node[m]["rdfid"]): 
                (a, b) = (k[0].n3(), k[1].n3())
                if "http://purl.bioontology.org/ontology/RXNORM/" in a: 
                    if a[1:len(a)-1] == "http://purl.bioontology.org/ontology/RXNORM/RXCUI": rxcuis.append(str(k[1]))
                    elif a[1:len(a)-1] == "http://purl.bioontology.org/ontology/RXNORM/RXAUI": rxauis.append(str(k[1]))
                    else:
                        etype = parseURI(a[1:len(a)-1])
                        if not etype in allowed_etypes: continue
                        lab = get_preferred_label(k[1], rxnorm)
                        lab = lab[0] if len(lab) > 0 else ""
                        if not opioid_drugs_ext.has_node(k[1].n3()):    
                            opioid_drugs_ext.add_node(k[1].n3(), ntype=extype, rdfid=k[1], label=lab)
                            cui_codes = get_cui_codes(k[1], rxnorm)
                            for n in cui_codes:
                                opioid_drugs_ext.add_node(str(n), rdfid=n, label=str(n), ntype="cui_id")
                                opioid_drugs_ext.add_edge(k[1].n3(), str(n), etype="has_cui")
                        opioid_drugs_ext.add_edge(m, k[1].n3(), etype=etype)
            opioid_drugs_ext.node[m]["rxcuis"] = ":-:".join(rxcuis)
            opioid_drugs_ext.node[m]["rxauis"] = ":-:".join(rxauis)
    return opioid_drugs_ext

def annotate_rxnorm_network(opioid_drugs, ntype="rxnorm_id_ext_1"):
    opioid_drugs_ext = deepcopy(opioid_drugs)
    for m in opioid_drugs.nodes():
        if opioid_drugs.node[m]["ntype"] == ntype: 
            rxcuis = []
            rxauis = []
            for k in rxnorm.predicate_objects(subject=opioid_drugs.node[m]["rdfid"]): 
                (a, b) = (k[0].n3(), k[1].n3())
                if "http://purl.bioontology.org/ontology/RXNORM/" in a: 
                    if a[1:len(a)-1] == "http://purl.bioontology.org/ontology/RXNORM/RXCUI": rxcuis.append(str(k[1]))
                    elif a[1:len(a)-1] == "http://purl.bioontology.org/ontology/RXNORM/RXAUI": rxauis.append(str(k[1]))
            opioid_drugs_ext.node[m]["rxcuis"] = ":-:".join(rxcuis)
            opioid_drugs_ext.node[m]["rxauis"] = ":-:".join(rxauis)
    return opioid_drugs_ext

#### ATC has different hierarchies related to Opioid Drugs
##### http://purl.bioontology.org/ontology/UATC/N07BC "Drugs used in opioid dependence"
##### http://purl.bioontology.org/ontology/UATC/N02A "Opioid Analgesics"
##### http://purl.bioontology.org/ontology/UATC/N01AH "Opioid Anasthetics" 
##### http://purl.bioontology.org/ontology/UATC/A06AH "Peripheral opioid receptor antagonists" (am retrieving these as well, can be excluded)
##### TODO: http://purl.bioontology.org/ontology/UATC/R05DA "Opium Alkaloids"

In [243]:
atc_hiercs = ["http://purl.bioontology.org/ontology/UATC/N02A", "http://purl.bioontology.org/ontology/UATC/N01AH", 
              "http://purl.bioontology.org/ontology/UATC/N07BC", "http://purl.bioontology.org/ontology/UATC/A06AH"]
opioid_drugs = get_opioid_drugs_atc(atc_hiercs)

In [244]:
for m in opioid_drugs.nodes():
    if not opioid_drugs.node[m]["ntype"] == "cui_id": continue
    a = get_rxnorm_ids(opioid_drugs.node[m]["rdfid"], rxnorm)
    for p in a: 
        opioid_drugs.add_node(p.n3(), rdfid=p, label=get_preferred_label(p, rxnorm)[0], ntype="rxnorm_id")
        opioid_drugs.add_edge(m, p.n3(), etype="has_rxnorm")

In [312]:
# available Edge types: set([u'Has Form', u'Rxn Strength', u'Constitutes', u'Rxn In Expressed Flag', u'Has Dose Form', 
#         u'Inverse Isa', u'Has Part', 'subclassOf', u'Part Of', u'Rxn Available Strength', u'Rxn Obsoleted', 
#         u'Rxn Quantity', u'Rxn Human Drug', u'Rxterm Form', u'Ndc', u'Has Doseformgroup', u'Has Precise Ingredient', 
#         u'Isa', 'has_cui', u'Rxn Activated', u'Ingredient Of', u'Ingredients Of', u'Consists Of', u'Has Ingredient', 
#         u'Precise Ingredient Of', u'Form Of', u'Has Tradename', 'has_rxnorm', u'Rxn Bn Cardinality'])
'''allowed_etypes = set([u'Has Form', u'Constitutes', u'Has Dose Form', u'Has Part', 'subclassOf', u'Part Of', 
                      u'Rxterm Form', u'Has Doseformgroup', u'Has Precise Ingredient', 'has_cui', u'Ingredient Of', 
                      u'Ingredients Of', u'Consists Of', u'Has Ingredient', u'Precise Ingredient Of', u'Form Of', 
                      u'Has Tradename', 'has_rxnorm'])
'''
### Ignore Has Part, Has Precise Ingredient, Has Ingredient, since "Acetaminophen /Codeine" has Codeine has an opioid, 
### but including these edges will retrieve CUIs for Acetaminophen which is not an opiate
### Ignore Has Dose Form, Has DoseFormGroup since e.g. instances are "Cartridge, Injectable Product, Pill, etc.
allowed_etypes = set([u'Has Form', u'Constitutes', 'subclassOf', u'Part Of', 
                      u'Rxterm Form', 'has_cui', u'Ingredient Of', u'Ingredients Of', 
                      u'Consists Of', u'Precise Ingredient Of', u'Form Of', u'Has Tradename', 'has_rxnorm'])

In [313]:
opioid_drugs_ext = extend_rxnorm_network(opioid_drugs)
opioid_drugs_ext = extend_rxnorm_network(opioid_drugs_ext, "rxnorm_id_ext", "rxnorm_id_ext_1")
opioid_drugs_ext = annotate_rxnorm_network(opioid_drugs_ext, "rxnorm_id_ext")

### Explore the Opioid Drug Network

In [256]:
def _get_node_set(G, node_type):
    node_set = []
    for m in G.nodes():
        if G.node[m]["ntype"] == node_type:
            node_set.append(m)
    return node_set

def _get_node_count(G, node_type):
    node_set = _get_node_set(G, node_type)
    return len(node_set)

def get_different_formulations(G, rxnorm_gen_node):
    e_groups = {}
    for k in G[rxnorm_gen_node]: 
        e = G[rxnorm_gen_node][k]["etype"]
        if not e in e_groups: e_groups[e] = [] 
        e_groups[e].append(G.node[k]["label"])
    for m in e_groups:
        print "----------"
        print m
        print "----------"
        for k in e_groups[m]: print k
            
def get_specific_network(G, rxnorm_gen_node):
    descendants = nx.descendants(G, rxnorm_gen_node)
    descendants = descendants.union(set([rxnorm_gen_node]))
    subG = nx.subgraph(G, list(descendants))
    return subG

In [314]:
print len(opioid_drugs_ext.nodes()), len(opioid_drugs_ext.edges())
#len(unopioid.nodes()), len(unopioid.edges())

3995 5160


In [315]:
etypes = set([])
for k in opioid_drugs_ext.edges(): 
    etype = opioid_drugs_ext[k[0]][k[1]]["etype"]
    etypes.add(etype)
print etypes

set([u'Ingredients Of', u'Has Form', 'subclassOf', u'Form Of', u'Part Of', u'Rxterm Form', u'Ingredient Of', u'Consists Of', u'Constitutes', u'Has Tradename', 'has_cui', 'has_rxnorm', u'Precise Ingredient Of'])


In [316]:
rxnorm_set = _get_node_set(opioid_drugs_ext, "rxnorm_id")
for k in rxnorm_set:
    print k, opioid_drugs_ext.node[k]["label"]

<http://purl.bioontology.org/ontology/RXNORM/3304> "Heroin"@en
<http://purl.bioontology.org/ontology/RXNORM/3290> "Dextromoramide"@en
<http://purl.bioontology.org/ontology/RXNORM/22713> "dezocine"@en
<http://purl.bioontology.org/ontology/RXNORM/236913> "LEVOMETHADONE"@en
<http://purl.bioontology.org/ontology/RXNORM/484259> "Ibuprofen / Oxycodone"@en
<http://purl.bioontology.org/ontology/RXNORM/1841> "Butorphanol"@en
<http://purl.bioontology.org/ontology/RXNORM/7676> "Opium"@en
<http://purl.bioontology.org/ontology/RXNORM/8119> "Phenazocine"@en
<http://purl.bioontology.org/ontology/RXNORM/1806700> "Naltrexone / Oxycodone"@en
<http://purl.bioontology.org/ontology/RXNORM/56795> "Sufentanil"@en
<http://purl.bioontology.org/ontology/RXNORM/7238> "Nalbuphine"@en
<http://purl.bioontology.org/ontology/RXNORM/8001> "Pentazocine"@en
<http://purl.bioontology.org/ontology/RXNORM/17933> "anileridine"@en
<http://purl.bioontology.org/ontology/RXNORM/8785> "Propoxyphene"@en
<http://purl.bioontology.or

In [317]:
#<http://purl.bioontology.org/ontology/RXNORM/7052>, Morphine
get_different_formulations(opioid_drugs_ext, "<http://purl.bioontology.org/ontology/RXNORM/7052>")

----------
Part Of
----------
"Atropine / Morphine"@en
"attapulgite / Morphine"@en
"Cyclizine / Morphine"@en
"guaiacolsulfonate / Morphine"@en
"Morphine / Naltrexone"@en
"bismuth subcarbonate / Morphine"@en
----------
Has Tradename
----------
"Infumorph"@en
"MS Contin"@en
"Morphabond"@en
"Embeda"@en
"Arymo"@en
"Astramorph"@en
"Avinza"@en
"Duramorph"@en
"Kadian"@en
----------
Ingredient Of
----------
"Morphine Oral Product"@en
"Morphine Sulfate 6.67 MG/ML"@en
"Morphine Sulfate 14.3 MG/ML"@en
"Morphine Rectal Suppository"@en
"Morphine Auto-Injector"@en
"Morphine Sulfate 20 MG/ML"@en
"Morphine Sulfate 150 MG"@en
"Morphine Injection"@en
"Morphine Sulfate 10 MG"@en
"Morphine Rectal Product"@en
"Morphine Sulfate 50 MG"@en
"Morphine Sulfate 4 MG/ML"@en
"Morphine Sulfate 30 MG"@en
"Morphine Injectable Product"@en
"Morphine Sulfate 2 MG/ML"@en
"Morphine Sulfate 6 MG/ML"@en
"Morphine Oral Tablet"@en
"Atropine / Morphine Injectable Product"@en
"Morphine Sulfate 45 MG"@en
"Morphine Sulfate 130 MG"

In [318]:
morphineG = get_specific_network(opioid_drugs_ext, "<http://purl.bioontology.org/ontology/RXNORM/7052>")
print len(morphineG.nodes()), len(morphineG.edges())

587 752


In [320]:
ap = nx.shortest_path(morphineG, "<http://purl.bioontology.org/ontology/RXNORM/7052>", "<http://purl.bioontology.org/ontology/RXNORM/203355>")
for a in range(len(ap)-1):
    print morphineG.node[ap[a]]["label"], morphineG[ap[a]][ap[a+1]]["etype"], morphineG.node[ap[a+1]]["label"]

"Morphine"@en Has Tradename "Duramorph"@en


### Saving and Printing the Opioid Drug Network

In [322]:
nx.write_gpickle(opioid_drugs_ext, "opioid_drugs_ext_main.gpickle")

In [325]:
def save_network_viz(G, file_name):
    #removing RDF uris
    aG = deepcopy(G)
    for k in aG.nodes(): aG.node[k]["rdfid"] = ""
    nx.write_gpickle(aG, file_name + ".gpickle")
    nx.write_graphml(aG, file_name + ".graphml") #printing purposes

In [326]:
save_network_viz(opioid_drugs_ext, "opioid_drugs_ext")

In [327]:
save_network_viz(morphineG, "morphineG")